# It's a one-to-one mapping version

In [1]:
from IPython.display import Image
from os import chdir
import numpy as np
import gudhi as gd
import matplotlib.pyplot as plt
import matplotlib

In [2]:
# read the file
file_name = "function_rips_with_threshold_100000_1"
#file_name="fig3-2"

rank_dim = 0

simplices = []
vertice_id = 0
i = 0

x_values = []
y_values = []

for line in open("../cgta_paper_2021/function_rips/"+file_name+".scc"):
#for line in open("./data/"+file_name+".txt"):
    if(line[0]!='#'):
        
        if(i<=1):
            i=i+1
            continue
            
        elif(i==2):
            rank_num=[] # sizes of generating sets
            for j in line.split():
                rank_num.append(int(j))
            i=i+1
            rank_num=rank_num[:-1]
            print("rank_num=",rank_num)
        else:
            block = line.split(';')
            grid = block[0].split()
            grid_x = float(grid[0])
            grid_y = float(grid[1])
            #grid_x = int(float(grid[0])*20)
            #grid_y = int(float(grid[1])*20)
            x_values.append(grid_x)
            y_values.append(grid_y)
            
            
            if len(block)>1:
                face = block[1].split()
                for j in range(len(face)):
                    face[j] = int(face[j])
                simplices.append([grid_x,grid_y,face])
                
            else:
                simplices.append([grid_x,grid_y,[vertice_id]])
                vertice_id += 1
            i=i+1   




simplex_tree = []
start_id=0
for i in range(len(rank_num)-1,-1,-1):
    simplices_i=[]
    for j in range(len(simplices)-start_id-rank_num[i],len(simplices)-start_id):
        simplices_i.append(simplices[j])
    start_id += rank_num[i]
    simplex_tree.append(simplices_i)
    


#transform face number to vertex number
for i in range(1,len(simplex_tree)):
    for j in range(len(simplex_tree[i])):
        simplex = simplex_tree[i][j]
        face = simplex[2]
        vertices=[]
        for face_id in face:
            vert = simplex_tree[i-1][face_id][2]
            for vert_id in vert:
                if vert_id not in vertices:
                    vertices.append(vert_id)
        simplex_tree[i][j][2]=vertices



## To verify the data structur
## simplices contain all the information we need
#print("the simplex tree:")
#for i in range(len(simplex_tree)):
#    for simplex in simplex_tree[i]:
#        print(simplex)   

print("len(simplex_tree)=",len(simplex_tree))
for i in range(len(simplex_tree)):
    print("length of simplex_tree",i,len(simplex_tree[i]))
       


rank_num= [37961, 36515, 20524, 5000]
len(simplex_tree)= 4
length of simplex_tree 0 5000
length of simplex_tree 1 20524
length of simplex_tree 2 36515
length of simplex_tree 3 37961


In [3]:

x_values_ordered = sorted(list({}.fromkeys(x_values).keys()))
y_values_ordered = sorted(list({}.fromkeys(y_values).keys()))


simplex_tree_integer=simplex_tree

for i in range(len(simplex_tree_integer)):
    for j in range(len(simplex_tree_integer[i])):
        simplex_tree_integer[i][j][0]=x_values_ordered.index(simplex_tree_integer[i][j][0])
        simplex_tree_integer[i][j][1]=y_values_ordered.index(simplex_tree_integer[i][j][1])




In [4]:
x_range=len(x_values_ordered)-1
y_range=len(y_values_ordered)-1

end_point = [x_range,y_range]
print("end_point ",end_point)

end_point  [20524, 4709]


## Compute rank invariant

In [5]:
def create_stair(elbow, end_point):
    # stair a path passing through the elbow.
    stair = []
    
    if(elbow[0]==0 or elbow[1]==end_point[1]):
        for i in range(end_point[1]+1):
            stair.append([0,i])
        for j in range(1,end_point[0]+1):
            stair.append([j,i])
    else:
        for i in range(elbow[1]+1):
            stair.append([0,i])   # i=0
        for j in range(elbow[0]):
            stair.append([j+1,i]) # j=0
        for k in range(i+1, end_point[1]+1):
            stair.append([j+1,k]) # k=2,3
        for l in range(j+2, end_point[0]+1):
            stair.append([l,k])
    return stair

def compute_rk(i,j,k,l,barcodes,rank_dim):
# This function computes the value rank(s,t) where s=(i,j) and t = (k,l)
    if(i>k or j>l):
        return 0
    
    s_1 = i + j
    t_1 = k + l
    rank0_inv = 0
    rank1_inv = 0

    for barcode in barcodes:
        if(barcode[0]==0):
            if(barcode[1][0]<=s_1 and barcode[1][1]>t_1):
                rank0_inv = rank0_inv + 1
        elif(barcode[0]==1):
            if(barcode[1][0]<=s_1 and barcode[1][1]>t_1):
                rank1_inv = rank1_inv + 1
    if(rank_dim==0):
        return rank0_inv
    elif(rank_dim==1):
        return rank1_inv
    

In [6]:
import time
#grid is from (0,0)
#vertex index is from 1
#rank_inv = np.array(np.zeros((x_range+1,y_range+1,x_range+1,y_range+1)))

st = gd.SimplexTree()
#result_rank_fn="./result/rank1_invariance_for_"+file_name+".txt"
#f_rank = f.open(result_rank_fn,"w")



#Scan all the elbows
for i in range(x_range+1):
    for j in range(y_range+1):
        elbow = [i,j]
        print("elbow",elbow)
        start_time=time.time()
        #print("time: start building a simplex tree",time.time())
        
        stair = create_stair(elbow, end_point)
        end_stair=time.time()
        print("time of creating a stair is ",end_stair-start_time,"s")
        #print("stair",stair)

        # Build the simplex tree
        
        for ii in range(len(simplex_tree_integer)): #4
            for jj in range(len(simplex_tree_integer[ii])): #[37961, 36515, 20524, 5000]
                print("simplex tree",ii,jj)
                start_loop = time.time()
                simplex = simplex_tree_integer[ii][jj]
                start_grid_id=time.time()
                for grid_id in stair:
                    if simplex[0]<=grid_id[0] and simplex[1]<=grid_id[1]:
                        filtration_value = grid_id[0]+grid_id[1] #TODO
                        start_insert=time.time()
                        #print("time enter insert/assign ", time.asctime( time.localtime(time.time())))
                        if((st.insert(simplex[2], filtration = filtration_value))==False): #TODO:optimize with C++, we don't have the funciton for Python
                            #print("time finish insert",time.asctime( time.localtime(time.time())))
                            end_insert=time.time()
                            st.assign_filtration(simplex[2], filtration = filtration_value)
                            end_assign=time.time()
                            print("assign time",end_assign-end_insert,"s")
                        else:
                            end_insert=time.time()
                        print("insert time",end_insert-start_insert,"s")
                        break
                end_loop = time.time()
                print("time for loop of stair ",end_loop-start_grid_id,"s")
                print("time for one loop of build a simplex tree",end_loop-start_loop,"s")
                # from 1.4e-05s to 0.0023s
                        
        st_time=time.time()                
        print("time of building a simplex tree",st_time-start_time,"s")
        print("simplices ",st.num_simplices())
        print("vertices ", st.num_vertices())
        #st_gen = st.get_filtration() 
        #for splx in st_gen :
        #    print(splx)
        
        barcodes = st.persistence()
        #print("barcodes,",barcodes)
        bc_time =time.time()
        print("time of calculating barcodes ",bc_time-st_time,"s")
        
        for start_id in range(len(stair)):
            for end_id in range(start_id,len(stair)):
                grid_start = stair[start_id]
                grid_end = stair[end_id]
                [ri,rj,rk,rl]=[grid_start[0],grid_start[1],grid_end[0],grid_end[1]]
                #print("[ri,rj,rk,rl]=",[ri,rj,rk,rl])
                #rank_inv[ri,rj,rk,rl]=compute_rk(ri,rj,rk,rl,barcodes,rank_dim)
                compute_rk(ri,rj,rk,rl,barcodes,rank_dim)
        ri_time = time.time()
        print("time of finishing calculating rank invariance",ri_time-bc_time,"s")
        print("total time for one elbow ",ri_time-start_time,"s")

        
                

elbow [0, 0]
time of creating a stair is  0.07013797760009766 s
simplex tree 0 0
insert time 3.409385681152344e-05 s
time for loop of stair  0.00011396408081054688 s
time for one loop of build a simplex tree 0.00011587142944335938 s
simplex tree 0 1
insert time 4.76837158203125e-06 s
time for loop of stair  6.29425048828125e-05 s
time for one loop of build a simplex tree 6.389617919921875e-05 s
simplex tree 0 2
insert time 3.0994415283203125e-06 s
time for loop of stair  5.602836608886719e-05 s
time for one loop of build a simplex tree 5.7220458984375e-05 s
simplex tree 0 3
insert time 4.0531158447265625e-06 s
time for loop of stair  0.0004658699035644531 s
time for one loop of build a simplex tree 0.00046706199645996094 s
simplex tree 0 4
insert time 5.0067901611328125e-06 s
time for loop of stair  3.0994415283203125e-05 s
time for one loop of build a simplex tree 3.314018249511719e-05 s
simplex tree 0 5
insert time 2.1457672119140625e-06 s
time for loop of stair  2.6941299438476562e-

insert time 5.245208740234375e-06 s
time for loop of stair  0.00010204315185546875 s
time for one loop of build a simplex tree 0.00010395050048828125 s
simplex tree 0 88
insert time 2.1457672119140625e-06 s
time for loop of stair  0.00022602081298828125 s
time for one loop of build a simplex tree 0.0002269744873046875 s
simplex tree 0 89
insert time 1.0967254638671875e-05 s
time for loop of stair  0.00012993812561035156 s
time for one loop of build a simplex tree 0.00013303756713867188 s
simplex tree 0 90
insert time 4.0531158447265625e-06 s
time for loop of stair  7.319450378417969e-05 s
time for one loop of build a simplex tree 7.510185241699219e-05 s
simplex tree 0 91
insert time 9.5367431640625e-07 s
time for loop of stair  7.486343383789062e-05 s
time for one loop of build a simplex tree 7.486343383789062e-05 s
simplex tree 0 92
insert time 4.0531158447265625e-06 s
time for loop of stair  9.202957153320312e-05 s
time for one loop of build a simplex tree 9.298324584960938e-05 s
sim

time for one loop of build a simplex tree 7.486343383789062e-05 s
simplex tree 0 160
insert time 6.198883056640625e-06 s
time for loop of stair  8.296966552734375e-05 s
time for one loop of build a simplex tree 8.511543273925781e-05 s
simplex tree 0 161
insert time 2.1457672119140625e-06 s
time for loop of stair  7.796287536621094e-05 s
time for one loop of build a simplex tree 7.891654968261719e-05 s
simplex tree 0 162
insert time 1.9073486328125e-06 s
time for loop of stair  7.486343383789062e-05 s
time for one loop of build a simplex tree 7.605552673339844e-05 s
simplex tree 0 163
insert time 1.9073486328125e-06 s
time for loop of stair  7.486343383789062e-05 s
time for one loop of build a simplex tree 7.557868957519531e-05 s
simplex tree 0 164
insert time 1.9073486328125e-06 s
time for loop of stair  0.0003807544708251953 s
time for one loop of build a simplex tree 0.0003807544708251953 s
simplex tree 0 165
insert time 2.1457672119140625e-06 s
time for loop of stair  7.605552673339

simplex tree 0 243
insert time 6.198883056640625e-06 s
time for loop of stair  0.00010609626770019531 s
time for one loop of build a simplex tree 0.00010800361633300781 s
simplex tree 0 244
insert time 4.0531158447265625e-06 s
time for loop of stair  9.512901306152344e-05 s
time for one loop of build a simplex tree 9.608268737792969e-05 s
simplex tree 0 245
insert time 9.5367431640625e-07 s
time for loop of stair  0.0004229545593261719 s
time for one loop of build a simplex tree 0.0004239082336425781 s
simplex tree 0 246
insert time 2.86102294921875e-06 s
time for loop of stair  0.00011277198791503906 s
time for one loop of build a simplex tree 0.00011372566223144531 s
simplex tree 0 247
insert time 9.5367431640625e-07 s
time for loop of stair  6.389617919921875e-05 s
time for one loop of build a simplex tree 6.508827209472656e-05 s
simplex tree 0 248
insert time 1.1920928955078125e-06 s
time for loop of stair  6.198883056640625e-05 s
time for one loop of build a simplex tree 6.1988830

insert time 3.0994415283203125e-06 s
time for loop of stair  0.0015571117401123047 s
time for one loop of build a simplex tree 0.0015571117401123047 s
simplex tree 0 374
insert time 2.86102294921875e-06 s
time for loop of stair  0.00011205673217773438 s
time for one loop of build a simplex tree 0.00011277198791503906 s
simplex tree 0 375
insert time 4.0531158447265625e-06 s
time for loop of stair  0.00016999244689941406 s
time for one loop of build a simplex tree 0.00017213821411132812 s
simplex tree 0 376
insert time 1.9073486328125e-06 s
time for loop of stair  0.00011110305786132812 s
time for one loop of build a simplex tree 0.00011181831359863281 s
simplex tree 0 377
insert time 1.1920928955078125e-06 s
time for loop of stair  0.00010800361633300781 s
time for one loop of build a simplex tree 0.00010895729064941406 s
simplex tree 0 378
insert time 1.9073486328125e-06 s
time for loop of stair  0.00011706352233886719 s
time for one loop of build a simplex tree 0.00011801719665527344

...

insert time 5.9604644775390625e-06 s
time for loop of stair  0.0018718242645263672 s
time for one loop of build a simplex tree 0.0018727779388427734 s
simplex tree 1 7047
insert time 3.814697265625e-06 s
time for loop of stair  0.0012006759643554688 s
time for one loop of build a simplex tree 0.0012018680572509766 s
simplex tree 1 7048
insert time 4.0531158447265625e-06 s
time for loop of stair  0.0018248558044433594 s
time for one loop of build a simplex tree 0.0018260478973388672 s
simplex tree 1 7049
insert time 5.9604644775390625e-06 s
time for loop of stair  0.0017418861389160156 s
time for one loop of build a simplex tree 0.0017418861389160156 s
simplex tree 1 7050
insert time 3.0994415283203125e-06 s
time for loop of stair  0.0007441043853759766 s
time for one loop of build a simplex tree 0.0007450580596923828 s
simplex tree 1 7051
insert time 6.198883056640625e-06 s
time for loop of stair  0.0009279251098632812 s
time for one loop of build a simplex tree 0.0009279251098632812 s

time for one loop of build a simplex tree 0.001817941665649414 s
simplex tree 1 7108
insert time 1.5974044799804688e-05 s
time for loop of stair  0.0022640228271484375 s
time for one loop of build a simplex tree 0.0022661685943603516 s
simplex tree 1 7109
insert time 1.4781951904296875e-05 s
time for loop of stair  0.0030460357666015625 s
time for one loop of build a simplex tree 0.0030469894409179688 s
simplex tree 1 7110
insert time 1.0967254638671875e-05 s
time for loop of stair  0.0010418891906738281 s
time for one loop of build a simplex tree 0.0010440349578857422 s
simplex tree 1 7111
insert time 7.867813110351562e-06 s
time for loop of stair  0.0016889572143554688 s
time for one loop of build a simplex tree 0.0016908645629882812 s
simplex tree 1 7112
insert time 8.106231689453125e-06 s
time for loop of stair  0.0014986991882324219 s
time for one loop of build a simplex tree 0.0014998912811279297 s
simplex tree 1 7113
insert time 8.821487426757812e-06 s
time for loop of stair  0.

insert time 9.059906005859375e-06 s
time for loop of stair  0.0031130313873291016 s
time for one loop of build a simplex tree 0.003113985061645508 s
simplex tree 1 7187
insert time 6.9141387939453125e-06 s
time for loop of stair  0.0021097660064697266 s
time for one loop of build a simplex tree 0.0021109580993652344 s
simplex tree 1 7188
insert time 6.9141387939453125e-06 s
time for loop of stair  0.0024619102478027344 s
time for one loop of build a simplex tree 0.0024628639221191406 s
simplex tree 1 7189
insert time 4.0531158447265625e-06 s
time for loop of stair  0.0013222694396972656 s
time for one loop of build a simplex tree 0.0013229846954345703 s
simplex tree 1 7190
insert time 5.0067901611328125e-06 s
time for loop of stair  0.0013439655303955078 s
time for one loop of build a simplex tree 0.001344919204711914 s
simplex tree 1 7191
insert time 8.821487426757812e-06 s
time for loop of stair  0.002708911895751953 s
time for one loop of build a simplex tree 0.0027098655700683594 s

insert time 4.0531158447265625e-06 s
time for loop of stair  0.0019130706787109375 s
time for one loop of build a simplex tree 0.0019140243530273438 s
simplex tree 1 7243
insert time 1.3113021850585938e-05 s
time for loop of stair  0.0021779537200927734 s
time for one loop of build a simplex tree 0.0021779537200927734 s
simplex tree 1 7244
insert time 1.5735626220703125e-05 s
time for loop of stair  0.003922939300537109 s
time for one loop of build a simplex tree 0.003924131393432617 s
simplex tree 1 7245
insert time 8.702278137207031e-05 s
time for loop of stair  0.004275083541870117 s
time for one loop of build a simplex tree 0.0042760372161865234 s
simplex tree 1 7246
insert time 1.3113021850585938e-05 s
time for loop of stair  0.0028569698333740234 s
time for one loop of build a simplex tree 0.0028579235076904297 s
simplex tree 1 7247
insert time 1.5020370483398438e-05 s
time for loop of stair  0.004408121109008789 s
time for one loop of build a simplex tree 0.004408836364746094 s


KeyboardInterrupt: 

In [ ]:
#Now it's rank0_invariance
np.save("./result/rank"+str(rank_dim)+"_invariance_for_"+file_name,rank_inv)

#for i in range(x_range+1):
#    for j in range(y_range+1):
#        for k in range(i,x_range+1):
#            for l in range(j,y_range+1):
#                print("rank(({0},{1}),({2},{3}))={4}".format(i,j,k,l,int(rank_inv[i,j,k,l])))


## compute multiplicity

In [ ]:
#
#m((i,j),(k,l))
def multp(rank, i,j,k,l):
    if(i<0 or j<0 or k>rank.shape[1] or l>rank.shape[0]):   
        return 0
    
    res = rank[i,j,k,l]
    #print("res=",res)
    if(k+1<rank.shape[0]):
        #print("rank[i,j,k+1,l]=",rank[i,j,k+1,l])
        res -= rank[i,j,k+1,l]
        #print("res=",res)
    if(l+1<rank.shape[1]):
        #print("rank[i,j,k,l+1]=",rank[i,j,k,l+1])
        res -= rank[i,j,k,l+1]
        #print("res=",res)
    if(k+1<rank.shape[0] and l+1<rank.shape[1]):
        #print("rank[i,j,k+1,l+1]=",rank[i,j,k+1,l+1])
        res += rank[i,j,k+1,l+1]
        #print("res=",res)
    #print("=================multp(({0},{1}),({2},{3}))={4}===================".format(i,j,k,l,res))
    return res
    

def compute_R_S_incl_excl(rank):
    multiplicity = np.zeros_like(rank)
    for i in range(rank.shape[0]):
        for j in range(rank.shape[1]):
            for k in range(i,rank.shape[0]):
                for l in range(j,rank.shape[1]):
                    m = multp(rank,i,j,k,l)-multp(rank,i-1,j,k,l)-multp(rank,i,j-1,k,l)+multp(rank,i-1,j-1,k,l)
                    if(m!=0):
                        multiplicity[i,j,k,l] = m
    return multiplicity
    

In [ ]:
rank_inv.shape

In [ ]:
m_start_time = time.time()
m = compute_R_S_incl_excl(rank_inv)

m_end_time = time.time()
print("time of calculating multiplicity:",m_end_time-m_start_time,"s")

np.save("./result/rank"+str(rank_dim)+"_multiplicity_for_"+file_name,m)

### Draw grid for multiplicity

In [ ]:
data = np.ones((x_range,y_range)) * np.nan

fig, ax = plt.subplots(1, 1, figsize=(10,10),tight_layout=True)
for x in range(x_range+1):
    ax.axvline(x, lw=1, color='k', zorder=5) # Horizontal 
for y in range(y_range+1):
    ax.axhline(y, lw=1, color='k', zorder=5) # Ordinate
print(m.shape)

for i in range(m.shape[0]):
    for j in range(m.shape[1]):
        for k in range(i,m.shape[0]):
            for l in range(j,m.shape[1]):
                if(m[i,j,k,l]):
                    print("m(({0},{1}),({2},{3}))={4}".format(i,j,k,l,int(m[i,j,k,l])))
                if(m[i,j,k,l]>0):
                    if(i==k and j==l):
                        ax.plot([i, k], [j, l],'bo', linestyle="-")
                    elif(i==k):
                        ax.plot([i+0.08, k+0.08], [j, l],'bv', linestyle="-")
                    elif(j==l):
                        ax.plot([i, k], [j+0.08, l+0.08],'bv',linestyle="-")
                    else:  
                        ax.plot([i, k], [j, l],'bv', linestyle="-")
                if(m[i,j,k,l]<0):
                    if(i==k and j==l):
                        ax.plot([i, k], [j, l],'ro', linestyle="-")
                    elif(i==k):
                        ax.plot([i-0.08, k-0.08], [j, l],'rv', linestyle="-")
                    elif(j==l):
                        ax.plot([i, k], [j-0.08, l-0.08],'rv', linestyle="-")
                    else:  
                        ax.plot([i, k], [j, l], 'rv', linestyle="-")

ax.imshow(data, interpolation='none', extent=[-1,x_range+1, -1, y_range+1], zorder=0)

#ax.axis('off')